# This file will be used to aggregate the output from the simulation ran in Transmodeler.

For that we will use the Vehicle Trajectories output with a resolution of 10 seconds.

I have been asked to produce the following outputs:
* Links **[Aggregated to the hour]**:
    * Per Vehicle Type:
        * Total Link Volume
        * Speed
    * Length
* Source Type **[Aggregated to the hour]**:
    * Per Vehicle Type and per link:
       * Fraction of the vehicle hours traveled in the link (fractions should sum to 1 for each link ID)
* Operation Mode Distribution **[Aggregated to the hour]**:
    * Per Link and Vehicle type:
        * Speed
        * Acceleration

In [1]:
import numpy as np
import pandas as pd

In [2]:
#Getting the output
Trajectories = pd.read_csv('data/Transmodeler_Output/Vehicle Trajectories.csv')

In [3]:
Trajectories.head()

,ID,Class,Time,Segment,Dir,Lane,Offset,Distance,Speed,Acceleration,Mileage,Heading,Longitude,Latitude
0,361115,Car Low MPR,28810.0,15107,1,2,0.0,0.820208,0.0000,0.00000,3.76590,184,-122336093,47590494
1,361116,Car Low MPR,28810.0,15107,1,3,0.0,0.820208,0.0000,0.00000,3.76539,184,-122336141,47590496
2,361117,Car Low MPR,28810.0,15107,1,2,0.0,23.961200,0.0000,0.00000,2.01032,184,-122336085,47590557
3,377096,Car Low MPR,28810.0,30887,0,2,NaN,-3.734290,36.3009,-6.64369,8.67427,356,-122335170,47593270
4,377095,Car Low MPR,28810.0,30887,0,2,0.0,162.089000,50.4904,-4.98565,9.32480,12,-122335216,47592811


# Getting the output for Links file

In [4]:
#It will be a dictionary where the keys are the segments
#Inside every segment there will be another couple of dictionaries for each mode of travel where the original data will be stored
segments = pd.unique(Trajectories['Segment'])
modes = pd.unique(Trajectories['Class'])
grouped = Trajectories.groupby(['Segment', 'Class'])
Links = {
    segment: {
        mode: grouped.get_group((segment, mode)).reset_index(drop=True)
        for mode in modes if (segment, mode) in grouped.groups
    }
    for segment in segments
}

In [5]:
#Getting the rundown on every link
Link_Output = pd.DataFrame(
    {
    'Segments': segment,
    'Class': mode
    }
    for mode in modes
    for segment in segments
)


In [6]:
pd.unique(Links[15107]['Car Low MPR']['ID']).shape[0]

474

In [7]:
for segment in segments:
    for mode in modes:
        if segment in Links and mode in Links[segment]:
            volume = pd.unique(Links[segment][mode]['ID']).shape[0]
            velocity = Links[segment][mode].groupby('ID')['Speed'].mean().mean()
            mask = (Link_Output['Segments'] == segment) & (Link_Output['Class'] == mode)
            Link_Output.loc[mask, 'Volume'] = volume
            Link_Output.loc[mask, 'Velocity'] = round(velocity,3)

In [8]:
Link_Output

,Segments,Class,Volume,Velocity
0,15107,Car Low MPR,474.0,13.612
1,30887,Car Low MPR,376.0,20.976
2,10214,Car Low MPR,849.0,46.904
3,17917,Car Low MPR,927.0,25.183
4,7,Car Low MPR,1061.0,31.015
...,...,...,...,...
79663,9219,DeliveryVan,NaN,NaN
79664,9950,DeliveryVan,NaN,NaN
79665,18843,DeliveryVan,NaN,NaN
79666,18419,DeliveryVan,NaN,NaN


# Source Type

In [9]:
SourceType_Output = pd.DataFrame(
    {
    'Segments': segment,
    'Class': mode
    }
    for mode in modes
    for segment in segments
)

In [10]:
for segment in segments:
    # Filter valid modes for the current segment
    valid_modes = {mode: Links[segment][mode] for mode in modes if segment in Links and mode in Links[segment]}

    # Precompute total_time just once
    total_time = sum(data.shape[0] * 10.0 for data in valid_modes.values())

    if total_time == 0:
        continue  # Avoid division by zero

    # Update fractions
    for mode, data in valid_modes.items():
        time = data['ID'].shape[0] * 10.0
        mask = (SourceType_Output['Segments'] == segment) & (SourceType_Output['Class'] == mode)
        SourceType_Output.loc[mask, 'Fraction'] = round(time / total_time, 3)

In [11]:
print(f'Sum of the fractions: {round(SourceType_Output[SourceType_Output['Segments'] == 4]['Fraction'].sum(),3)}')
SourceType_Output

Sum of the fractions: 1.0


,Segments,Class,Fraction
0,15107,Car Low MPR,0.900
1,30887,Car Low MPR,0.711
2,10214,Car Low MPR,0.850
3,17917,Car Low MPR,0.867
4,7,Car Low MPR,0.900
...,...,...,...
79663,9219,DeliveryVan,NaN
79664,9950,DeliveryVan,NaN
79665,18843,DeliveryVan,NaN
79666,18419,DeliveryVan,NaN


# Operation Mode Distribution

In [12]:
#Getting the categories
Operation_Categories = {} #From the moves datafiles
Operation_Categories[11] = {
    'VSP_min': -999999,
    'VSP_max': 0,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[12] = {
    'VSP_min': 0,
    'VSP_max': 3,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[13] = {
    'VSP_min': 3,
    'VSP_max': 6,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[14] = {
    'VSP_min': 6,
    'VSP_max': 9,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[15] = {
    'VSP_min': 9,
    'VSP_max': 12,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[16] = {
    'VSP_min': 12,
    'VSP_max': 9999999,
    'Speed_min': 1,
    'Speed_max': 25,
}
Operation_Categories[21] = {
    'VSP_min': -9999,
    'VSP_max': 0,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[22] = {
    'VSP_min': 0,
    'VSP_max': 3,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[23] = {
    'VSP_min': 3,
    'VSP_max': 6,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[24] = {
    'VSP_min': 6,
    'VSP_max': 9,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[25] = {
    'VSP_min': 9,
    'VSP_max': 12,
    'Speed_min': 25,
    'Speed_max': 50,
}
# Operation_Categories[26] = {
#     'VSP_min': 12,
#     'VSP_max': 999999,
#     'Speed_min': 25,
#     'Speed_max': 50,
# }
Operation_Categories[27] = {
    'VSP_min': 12,
    'VSP_max': 18,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[28] = {
    'VSP_min': 18,
    'VSP_max': 24,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[29] = {
    'VSP_min': 24,
    'VSP_max': 30,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[30] = {
    'VSP_min': 30,
    'VSP_max': 999999,
    'Speed_min': 25,
    'Speed_max': 50,
}
Operation_Categories[33] = {
    'VSP_min': -999999,
    'VSP_max': 6,
    'Speed_min': 50,
    'Speed_max': 99999,
}
Operation_Categories[35] = {
    'VSP_min': 6,
    'VSP_max': 12,
    'Speed_min': 50,
    'Speed_max': 99999,
}
# Operation_Categories[36] = {
#     'VSP_min': 12,
#     'VSP_max': 999999,
#     'Speed_min': 50,
#     'Speed_max': 99999,
# }
Operation_Categories[37] = {
    'VSP_min': 12,
    'VSP_max': 18,
    'Speed_min': 50,
    'Speed_max': 99999,
}
Operation_Categories[38] = {
    'VSP_min': 18,
    'VSP_max': 24,
    'Speed_min': 50,
    'Speed_max': 99999,
}
Operation_Categories[39] = {
    'VSP_min': 24,
    'VSP_max': 30,
    'Speed_min': 50,
    'Speed_max': 99999,
}
Operation_Categories[40] = {
    'VSP_min': 30,
    'VSP_max': 99999,
    'Speed_min': 50,
    'Speed_max': 99999,
}

In [13]:
# Defining the A, B, C and mass value for the cars using this link
# https://www.fhwa.dot.gov/ENVIRonment/air_quality/conformity/research/project_level_analyses/pla02.cfm
VSP_data = {
    'Cars': {
        'A': 0.156461,
        'B': 0.00200193,
        'C': 0.000492646,
        'm': 1.4788
    },
    'Trucks': {
        'A': 2.08126,
        'B': 0,
        'C': 0.00418844,
        'm': 31.4038
    }
}
Mode_to_VSP = {
    'Cars': ['Car Low MPR', 'Car Mid MPR', 'Pickup/SUV',
       'Car High MPR', 'Motorcycle', 'Bus',
       'DeliveryVan'],
    'Trucks': ['SU Truck', 'Trailer Truck']
}

In [15]:
speed = Trajectories['Speed']
accel = Trajectories['Acceleration']
v_class = np.where(Trajectories['Class'].isin(Mode_to_VSP['Cars']), 'Cars', 'Trucks')

A = [VSP_data[c]['A'] for c in v_class]
B = [VSP_data[c]['B'] for c in v_class]
C = [VSP_data[c]['C'] for c in v_class]
mass = [VSP_data[c]['m'] for c in v_class]

Trajectories['VSP'] = (np.array(A)*speed + np.array(B)*speed**2 + np.array(C)*speed**3 + np.array(mass)*speed*accel) / np.array(mass)


In [16]:
def get_category(row):
    for cat, props in Operation_Categories.items():
        if (props['VSP_min'] <= row['VSP'] < props['VSP_max'] and
            props['Speed_min'] <= row['Speed'] < props['Speed_max']):
            return cat
    return np.nan

Trajectories['Category'] = Trajectories.apply(get_category, axis=1)

In [17]:
Trajectories.head()

,ID,Class,Time,Segment,Dir,Lane,Offset,Distance,Speed,Acceleration,Mileage,Heading,Longitude,Latitude,VSP,Category
0,361115,Car Low MPR,28810.0,15107,1,2,0.0,0.820208,0.0000,0.00000,3.76590,184,-122336093,47590494,0.000000,NaN
1,361116,Car Low MPR,28810.0,15107,1,3,0.0,0.820208,0.0000,0.00000,3.76539,184,-122336141,47590496,0.000000,NaN
2,361117,Car Low MPR,28810.0,15107,1,2,0.0,23.961200,0.0000,0.00000,2.01032,184,-122336085,47590557,0.000000,NaN
3,377096,Car Low MPR,28810.0,30887,0,2,NaN,-3.734290,36.3009,-6.64369,8.67427,356,-122335170,47593270,-219.611338,21.0
4,377095,Car Low MPR,28810.0,30887,0,2,0.0,162.089000,50.4904,-4.98565,9.32480,12,-122335216,47592811,-200.054626,33.0


In [18]:
time_lookup = Trajectories.groupby(['Segment', 'Class']).size().mul(10).rename('Total Time').reset_index()
time_lookup_wCategory = Trajectories.groupby(['Segment', 'Class', 'Category']).size().mul(10).rename('Time').reset_index()
time_lookup_wCategory = pd.merge(time_lookup_wCategory, time_lookup, on=['Segment', 'Class'])
time_lookup_wCategory['Fraction'] = round(time_lookup_wCategory['Time'] / time_lookup_wCategory['Total Time'], 4)
time_lookup_wCategory

,Segment,Class,Category,Time,Total Time,Fraction
0,4,Car High MPR,30.0,10,150,0.0667
1,4,Car High MPR,40.0,140,150,0.9333
2,4,Car Low MPR,21.0,30,3660,0.0082
3,4,Car Low MPR,30.0,50,3660,0.0137
4,4,Car Low MPR,33.0,110,3660,0.0301
...,...,...,...,...,...,...
197339,31353,SU Truck,11.0,20,90,0.2222
197340,31353,SU Truck,12.0,10,90,0.1111
197341,31353,SU Truck,14.0,10,90,0.1111
197342,31353,SU Truck,16.0,30,90,0.3333


In [19]:
Link_Output.to_csv('Link_Output.csv', index=False)
SourceType_Output.to_csv('SourceType_Output.csv', index=False)
time_lookup_wCategory.to_csv('time_lookup_wCategory.csv', index=False)